# 0: DataDataData

Here's a sample class wrapping the main abstraction, the `Dataset`.

* TODO Support for multiple users? @ben

In [98]:
from collections import Counter
import csv
import random

from sklearn.datasets import fetch_20newsgroups


class Dataset(object):
    """ Encapsulates unlabelled and labelled examples. """
    def __init__(self, text_to_label=None):
        self.text_to_label = text_to_label or {}
        
    def add_label(self, text, label):
        self.text_to_label[text] = label
        
    def __iter__(self):
        return ((text, label) for text, label in
                 self.text_to_label.items())
    
    def to_csv(self, fname):
        with open(fname, 'w') as f:
            w = csv.writer(f, delimiter=',')
            for text, label in self.text_to_label.items():
                w.writerow((label, text))
    
    @classmethod
    def from_csv(cls, fname):
        with open(fname) as f:
            return cls({text: label for text, label in
                        csv.reader(f, delimiter=',')})
        
    def update(self, other):
        self.text_to_label.update(other.text_to_label)
    
    @property
    def label_distribution(self):
        return dict(Counter(self.text_to_label.values()))
    

def build_newsgroups(subset='train'):
    """ Fetches newsgroupd data and returns a Dataset. """
    newsgroups_train = fetch_20newsgroups(subset=subset)
    label_names = {index: name for index, name in 
                   enumerate(newsgroups_train.target_names)}
    # Transform to guns or not.
    for i, name in list(label_names.items()):
        label_names[i] = name == 'talk.politics.guns'
    return Dataset({text: label_names[index] 
                    for text, index in zip(newsgroups_train.data, 
                                           newsgroups_train.target)})

def unlabel(dataset, p=0.01):
    """ Randomly removes some labels. """
    for text, label in dataset:
        if random.random() > p:
            dataset.add_label(text, None)


train = build_newsgroups()
unlabel(train)
print(train.label_distribution)

{None: 11196, False: 111, True: 7}


# 1.1 Annotation & reliability

* TODO Learning curve @ben
* TODO Agreement @ben


## Samplers

For example, a random unlabelled sampler.

In [99]:
import random


class Random(object):
    def __init__(self, limit=None):
        self.limit = limit
        
    def __call__(self, dataset):
        unlabelled = list(((text, label) 
                          for (text, label) in dataset 
                          if label is None))
        random.shuffle(unlabelled)
        yielded = 0
        for text, label in unlabelled:
            yield text, label
            yielded += 1
            if self.limit and yielded == self.limit:
                break

## Annotation tool

In [103]:
from IPython.display import clear_output, display
import ipywidgets as widgets
import functools


class AnnotationPane(object):
    LABELS = [
        ('Yes', True),
        ('No', False),
    ]
    def __init__(self, dataset, sampler):
        self.text_pane = widgets.HTML()
        self.buttons = []
        for desc, label in self.LABELS:
            button = widgets.Button(
                description=desc
            )
            button.on_click(functools.partial(self.on_click, label))
            self.buttons.append(button)
        
        self.view = widgets.VBox([
            widgets.HBox(self.buttons),
            self.text_pane,  
        ])
        self.queue = [text for text, label in sampler(dataset)]
        self.dataset = dataset
        self.draw()
        display(self.view)   

    def on_click(self, label, button):
        self.dataset.add_label(self.text, label)
        self.draw()
        
    def draw(self):
        if not self.queue:
            for button in self.buttons:
                button.disabled = True
            self.text_pane.value = '<p>Finished</p>'
        else:
            self.text = self.queue.pop(0)
            self.text_pane.value = '<p>{}</p>'.format(self.text)

## Manually label some examples

In [104]:
pane = AnnotationPane(train, Random(10))

VBox(children=(HBox(children=(Button(description='Yes', style=ButtonStyle()), Button(description='No', style=ButtonStyle()))), HTML(value='<p>From: c5ff@jupiter.sun.csd.unb.ca (COOK  Charlie)\nSubject: NHL Summary parse results for games played Sun, April 18, 1993\nOrganization: University of New Brunswick\nLines: 219\n\nNew Jersey                       1 0 2--3\nPittsburgh                       2 3 1--6\nFirst period\n     1, Pittsburgh, Tocchet 1 (Stevens, Lemieux) pp, 1:40.\n     2, New Jersey, Barr 1 (Guerin, Holik) 6:24.\n     3, Pittsburgh, Lemieux 1 (Jagr, Ramsey) 9:33.\nSecond period\n     4, Pittsburgh, Lemieux 2 (Stevens, Murphy) pp, 4:11.\n     5, Pittsburgh, Francis 1 (Ramsey, Mullen) 12:57.\n     6, Pittsburgh, Tippett 1 (Jagr, McEachern) 17:13.\nThird period\n     7, Pittsburgh, Jagr 1 (Samuelsson, Lemieux) pp, 8:35.\n     8, New Jersey, Stevens 1 (Niedermayer, Driver) pp, 11:48.\n     9, New Jersey, Stevens 2 (Semak, Niedermayer) 18:56.\n\nPittsburgh: 6    Power play: 8-3\nScorer            G    A   Pts\n---------------  ---  ---  ---\nFrancis            1    0    1\nJagr               1    2    3\nLemieux            2    2    4\nMcEachern          0    1    1\nMullen             0    1    1\nMurphy             0    1    1\nRamsey             0    2    2\nSamuelsson         0    1    1\nStevens            0    2    2\nTippett            1    0    1\nTocchet            1    0    1\n\nNew Jersey: 3    Power play: 8-1\nScorer            G    A   Pts\n---------------  ---  ---  ---\nBarr               1    0    1\nDriver             0    1    1\nGuerin             0    1    1\nHolik              0    1    1\nNiedermayer        0    2    2\nSemak              0    1    1\nStevens            2    0    2\n\n-----------------------------------------\nSt Louis                         0 2 2--4\nChicago                          1 2 0--3\nFirst period\n     1, Chicago, Noonan 1 (Larmer, Brown) 8:17.\nSecond period\n     2, St Louis, Brown 1 (Shanahan, Emerson) 3:12.\n     3, Chicago, Noonan 2 (Roenick, Chelios) pp, 5:40.\n     4, Chicago, Noonan 3 (Matteau, Sutter) 8:51.\n     5, St Louis, Felsner 1 (McRae, Janney) 12:49.\nThird period\n     6, St Louis, Shanahan 1 (Brown, Hull) pp, 11:12.\n     7, St Louis, Hull 1 (Emerson, Brown) pp, 11:29.\n\nSt Louis: 4    Power play: 4-2\nScorer            G    A   Pts\n---------------  ---  ---  ---\nBrown              1    2    3\nEmerson            0    2    2\nFelsner            1    0    1\nHull               1    1    2\nJanney             0    1    1\nMcRae              0    1    1\nShanahan           1    1    2\n\nChicago: 3    Power play: 7-1\nScorer            G    A   Pts\n---------------  ---  ---  ---\nBrown              0    1    1\nChelios            0    1    1\nLarmer             0    1    1\nMatteau            0    1    1\nNoonan             3    0    3\nRoenick            0    1    1\nSutter             0    1    1\n\n-----------------------------------------\nLos Angeles                      1 3 2--6\nCalgary                          0 1 2--3\nFirst period\n     1, Los Angeles, Sydor 1 (Gretzky, Sandstrom) 0:16.\nSecond period\n     2, Calgary, Suter 1 (Fleury) sh, 2:48.\n     3, Los Angeles, Carson 1 (Shuchuk, Sydor) pp, 3:13.\n     4, Los Angeles, Huddy 1 (Taylor, Rychel) 3:37.\n     5, Los Angeles, McSorley 1 (unassisted) 6:36.\nThird period\n     6, Los Angeles, Millen 1 (Granato, Donnelly) 1:06.\n     7, Calgary, Dahlquist 1 (Otto) 4:23.\n     8, Calgary, Yawney 1 (MacInnis, Reichel) 8:47.\n     9, Los Angeles, Carson 2 (Sandstrom, Robitaille) pp, 10:32.\n\nLos Angeles: 6    Power play: 10-2\nScorer            G    A   Pts\n---------------  ---  ---  ---\nCarson             2    0    2\nDonnelly           0    1    1\nGranato            0    1    1\nGretzky            0    1    1\nHuddy              1    0    1\nMcSorley           1    0    1\nMillen             1    0    1\nRobitaille         0    1    1\nRychel             0    1    1\nSandstrom          0  

## See our new labels in the dataset

In [105]:
print(train.label_distribution)

{None: 11176, False: 121, True: 17}


# 1.1: Active learning

Here is a straw man active sampler that:
* trains a classifier on the labelled data
* predicts the labels of unlabelled data
* selects text with a specific label profile

In [113]:
from collections import Counter

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline


def train_classifier(dataset, cv=3):
    X, y = [], []
    for text, label in dataset:
        assert isinstance(text, str)
        assert label in {True, False}
        X.append(text)
        y.append(label)
    print('Got {} labelled samples'.format(len(X)))
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('clf', SGDClassifier(loss='log')),
    ])
    if cv:
        print('Cross-validating')
        scores = cross_val_score(pipeline, X, y, cv=cv)
        print("Cross-validated accuracy: %0.2f (+/- %0.2f)" %
              (scores.mean(), scores.std() * 2))
    print('Refitting')
    pipeline.fit(X, y)
    return pipeline


class Active(object):
    def __init__(self, pipeline, 
                 query=lambda i: i[1] is None, 
                 accept=lambda i: True, limit=None):
        self.pipeline = pipeline
        self.limit = limit
        self.query = query
        self.accept = accept
    
    def __call__(self, dataset):
        X, y = [], []
        for text, label in dataset:
            if self.query((text, label)):
                X.append(text)
                y.append(label)
        print('Predicting {} unlabelled'.format(len(X)))
        yielded = 0
        for probs, text in zip(self.pipeline.predict_proba(X), X):
            predictions = dict(zip(self.pipeline.classes_, probs))
            if self.accept(predictions):
                yield text, predictions
                yielded += 1
                if self.limit and yielded == self.limit:
                    break

In [114]:
labelled = ((text, label) for text, label in train if label in {True, False})
pipeline = train_classifier(labelled, cv=3)
pane = AnnotationPane(train, Active(pipeline, 
                                    accept=lambda pred: pred[True] > 0.8,
                                    limit=10))

Got 138 labelled samples
Cross-validating


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will b

Cross-validated accuracy: 0.84 (+/- 0.06)
Refitting


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Predicting 11176 unlabelled


/Users/wradford/repos/personal/alta17active/.venv/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


VBox(children=(HBox(children=(Button(description='Yes', style=ButtonStyle()), Button(description='No', style=ButtonStyle()))), HTML(value='<p>From: km@cs.pitt.edu (Ken Mitchum)\nSubject: Re: Patient-Physician Diplomacy\nArticle-I.D.: pitt.19422\nReply-To: km@cs.pitt.edu (Ken Mitchum)\nOrganization: Univ. of Pittsburgh Computer Science\nLines: 22\n\nIn article <C4Hyou.1Iz@mentor.cc.purdue.edu> hrubin@pop.stat.purdue.edu (Herman Rubin) writes:\n>In article <188@ky3b.UUCP> km@ky3b.pgh.pa.us (Ken Mitchum) writes:\n>\n>>Ditto. Disease is a great leveling experience, however. Some people\n>>are very much afronted to find out that all the money in the world\n>>does not buy one health. Everyone looks the same when they die.\n>\n>If money does not buy one health, why are we talking about paying\n>for medical expenses for those not currently "adequately covered"?\n\nHerman, I would think you of all people would/could distinguish\nbetween "health" and "treatment of disease." All the prevention\nmedicine people preach this all the time. You cannot buy health.\nYou can buy treatment of disease, assuming you are lucky enough\nto have a disease which can be treated. A rich person with a\nterminal disease is a bit out of luck. There is no such thing\nas "adequately covered" and there never will be. \n\nAnd for what it\'s worth, I\'ll be the first to admit that all my\npatients die.\n\n-km\n</p>')))

In [115]:
print(train.label_distribution)

{None: 11146, False: 137, True: 31}


# 2: Live shared task

We can weakly supervise using precise functions. Note that we have no fancy model above the labelling functions.

* TODO Incorporate `snorkel` @ben
* TODO Setup task @will
  * Create `/submisssions/$USER`
  * Copy notebook template
  * Load shared task data pool.
* TODO Push task @will
  * Save data to `.csv`
  * `git commit/push`